# Logistic Regression

### 推导过程
考虑具有 $n$ 个独立变量的向量 $x=(x_1,x_2,...,x_n)$ ，设条件概率 $P(y=1|x)=p$ 为根据观测量相对于某事件 $x$ （即输入）发生的概率，则 Logistic 回归模型可以表示为：

$$P(y=0|x)=1-P(y=1|x)=1-\frac{1}{1+e^{-g(x)}}=\frac{1}{1+e^{g(x)}}$$

这里 $f(x)=\frac{1}{1+e^{-x}}$ 称为 Logistic 函数，其中 $g(x)=w_0+w_1x_1+...+w_nx_n$ 。

注意：参数向量是要比输入向量多一个维度的，为了规范化。

那么在 $x$ 条件下 $y$ 不发生的概率为：

$$P(y=0|x)=1-P(y=1|x)=1-\frac{1}{1+e^{-g(x)}}=\frac{1}{1+e^{g(x)}}$$

所以在 $x$ 条件下事件发生与不发生的概率之比为：

$$\frac{P(y=1|x)}{P(y=0|x)}=\frac{p}{1-p}=e^{g(x)}$$

这个比值称为事件的发生比（ the odds of experiencing an event ），简记为 odds。

对 odds 取对数得到：

$$ln(\frac{p}{1-p})=g(x)=w_0+w_1x_1+...+w_nx_n$$

核心：使用最大似然估计求参数。

假设有 $m$ 个观测样本，观测值分别为 $y_1,y_2,...,y_m$ ，设 $p_i=P(y_i=1|x_i)$ 为给定条件下得到 $y_i=1$ 的概率，同样地， $y_i=0$ 的概率为 $P(y_i=0|x_i)=1-p_i$ ，所以得到一个观测值的概率为 $P(y_i)=p_i^{y_i}(1-p_i)^{1-y_i}$ 。

因为各个观测样本之间互相独立，那么它们的联合分布为各边缘分布的乘积。得到似然函数为：

$$L(w)=\prod_{i=1}^{m}(\pi(x_i)^{y_i})(1-\pi(x_i))^{1-y_i}$$
				
目标是求出使这一似然函数的值最大的参数估计，最大似然估计就是求出参数 $w_0,w_1,...,w_n$ ，使得 $L(w)$ 取得最大值，对函数 $L(w)$ 取对数得到：

$$lnL(w)=\sum_{i=1}^{m}(y_iln[\pi(x_i)]+(1-y_i)ln[1-\pi(x_i)])$$

继续对这 $n+1$ 个 $w_i$ 分别求偏导，得到 $n+1$ 个方程，比如现在对参数 $w_k$ 求偏导，由于：

$(y_iln[\pi(x_i)]+(1-y_i)ln[1-\pi(x_i)])^{'}$

$=\frac{y_i}{\pi(x_i)}[\pi(x_i)]^{'}+(1-y_i)\frac{-[\pi(x_i)]^{'}}{1-\pi(x_i)}$

$=[\frac{y_i}{\pi(x_i)}-\frac{1-y_i}{1-\pi(x_i)}][\pi(x_i)]^{i}$

$=(y_i-\pi(x_i))g^{'}(x)$

$=x_{ik}[y_i-\pi(x_i)]$

所以，

$$\frac{\partial lnL(w_k)}{\partial w_k}=\sum_{i=1}^{m}x_{ik}[y_i-\pi(x_i)]=0$$
				
这样的方程一共有 $n+1$ 个，所以现在的问题转化为解这 $n+1$ 个方程形成的方程组。

### 相关问题和概念

#### 问题：什么是最大似然估计（极大似然估计）？
首先理解似然：通过观测值反推概率，就是似然。

举例：在理想情况下，抛硬币正反面的概率都分别为 0.5 ，但如果扔了 1000 次硬币，都出现了“花”，那么就会反推这个概率不是公平的。在这里，把硬币出现“花”出现的概率称为硬币的参数。

上述问题的回答：得到最可能的参数，叫做最大似然估计。

#### 概念：概率和似然
概率：已知硬币的参数，就可以去推测抛硬币的各种情况的可能性。

似然：不知道硬币的参数，通过抛硬币的结果去推测硬币的参数。

#### 问题：如何进行最大似然估计？
核心：求似然函数的问题，变成求似然函数的极值。

举例：假设硬币的参数，然后计算实验结果的概率是多少，概率越大的，那么这个假设的参数就越可能是真的。

#### 概念：牛顿迭代法、梯度下降法和最小二乘法

#### 问题：什么是一阶收敛？什么是  阶收敛？
核心：$n$ 阶收敛相当于以 $n$ 阶的速度逼近一个值 $x$ 。

一阶收敛：以 $\frac{1}{n}$ 的速度收敛；（梯度上升法）

二阶收敛：以 $\frac{1}{n^2}$ 的速度收敛。（牛顿迭代法）

结论：二阶收敛比一阶收敛速度更快。

### Python单机朴素实现

In [1]:
import numpy as np
import math

class Sigmoid():
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))

    def gradient(self, x):
        return self.__call__(x) * (1 - self.__call__(x))

class LogisticRegression():
    """ Logistic Regression classifier.
    Parameters:
    -----------
    learning_rate: float
        The step length that will be taken when following the negative gradient during
        training.
    """
    def __init__(self, learning_rate=.1):
        self.param = None
        self.learning_rate = learning_rate
        self.sigmoid = Sigmoid()

    def _initialize_parameters(self, X):
        n_features = np.shape(X)[1]
        # Initialize parameters between [-1/sqrt(N), 1/sqrt(N)]
        limit = 1 / math.sqrt(n_features)
        self.param = np.random.uniform(-limit, limit, (n_features,))

    def fit(self, X, y, n_iterations):
        self._initialize_parameters(X)
        # Tune parameters for n iterations
        for i in range(n_iterations):
            # Make a new prediction
            y_pred = self.sigmoid(X.dot(self.param))
            self.param -= self.learning_rate * -(y - y_pred).dot(X)

    def predict(self, X):
        y_pred = np.round(self.sigmoid(X.dot(self.param)))
        return y_pred.astype(int)

### TensorFlow 实现

In [ ]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", “{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

### CUDA 实现

### Hadoop 实现

### Spark 实现

### Parameter Server 实现